In [39]:
%pip install nltk
import nltk
from nltk.corpus import wordnet as wn, stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/eliotullmo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/eliotullmo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/eliotullmo/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Lets try to implement a simple algorithm based on this paper: key to crossword solving

In [74]:
def modified_lesk(sent, k):
    text_words = word_tokenize(sent)
    stop_words = set(stopwords.words("english"))
    temp_words = list(set(text_words) - set(stop_words))
    if len(temp_words) != 0:
        text_words = temp_words
    word_synsets = []
    for i, val in enumerate(text_words):
        word_synsets.extend(wn.synsets(val))

    output_lesk = {}
    context_words = []

    for i, val in enumerate(word_synsets):
        words = word_tokenize(val.definition())
        context_words.extend(words)

    context_words = list(set(context_words) - set(stop_words))
    print(context_words)
    print("done")
    for word in list(wn.all_lemma_names(lang='eng')):
        if len(word) == k and word.isalpha():
            lesk = 0
            for synset in list(wn.synsets(word)):
                def_words = word_tokenize(synset.definition())
                def_words = list(set(def_words) - set(stop_words))
                temp1 = set(text_words).intersection(def_words)
                temp2 = set(context_words).intersection(def_words)
                temp1 = len(temp1) / len(text_words)
                temp2 = len(temp2) / len(context_words)
                t = 0.75 * temp1 + 0.25 * temp2
                lesk = max(t, lesk)
            if lesk > 0:
                output_lesk[word] = lesk

    rank = list(sorted(output_lesk, key=output_lesk.__getitem__, reverse=True))
    return rank[:1000]


In [75]:
modified_lesk("french speaking country ", 6)


[')', 'colonized', 'France', 'cooking', 'indefinite', 'pertaining', 'language', 'make', 'cities', 'body', 'people', 'countries', 'outside', 'beans', 'public', 'Washington', 'sculptor', 'region', 'talk', 'capable', 'States', 'purpose', 'involving', 'utterance', 'delivering', 'live', 'usually', 'characteristic', 'boundary', 'area', 'Romance', 'special', 'audience', 'use', 'serving', 'single', 'exchange', 'address', 'occupied', '1850-1931', 'speech', ';', 'e.g', 'geographical', 'nation', 'express', 'United', 'culture', 'cut', 'politically', 'particular', 'territory', 'geography', 'Abraham', ',', 'intelligible', 'created', 'seated', 'spoken', 'thoughts', 'natural', 'Lincoln', 'D.C.', 'figure', 'speaking', 'marble', 'government', 'give', 'country', 'organized', 'Memorial', '(', 'lengthwise', 'preparation', 'distinguished', 'towns', 'sound']
done


['brasil',
 'brazil',
 'estate',
 'tweedy',
 'arroba',
 'cathay',
 'colony',
 'defect',
 'desert',
 'greece',
 'nation',
 'russia',
 'thrace',
 'soviet',
 'tinpot',
 'bharat',
 'chorus',
 'cyprus',
 'export',
 'fealty',
 'import',
 'manchu',
 'source',
 'sultan',
 'tungus',
 'altaic',
 'folksy',
 'native',
 'rustic',
 'amhara',
 'barrio',
 'belize',
 'canaan',
 'canton',
 'consul',
 'france',
 'people',
 'prague',
 'salish',
 'toltec',
 'watusi',
 'zurich',
 'wander',
 'abroad',
 'regent',
 'upland',
 'mutely',
 'armory',
 'emigre',
 'mbundu',
 'muzzle',
 'pathan',
 'rapine',
 'squire',
 'tirana',
 'tongue',
 'townee',
 'deport',
 'french',
 'county',
 'romani',
 'romany',
 'bomber',
 'busker',
 'calder',
 'disney',
 'hoagie',
 'keller',
 'rhodes',
 'talker',
 'truman',
 'varese',
 'echoic',
 'formal',
 'magyar',
 'nepali',
 'sacred',
 'access',
 'albers',
 'apache',
 'arnold',
 'bartok',
 'benton',
 'bihari',
 'carson',
 'carver',
 'chavez',
 'cooper',
 'coyote',
 'dubyuh',
 'edison',

In [76]:
import nltk
from nltk.corpus import wordnet as wn, stopwords
from nltk.tokenize import word_tokenize


def noun_fn(sent,a,b):
    text_words = word_tokenize(sent)
    stop_words = set(stopwords.words("english"))
    temp_words = list(set(text_words) - set(stop_words))
    if len(temp_words) != 0:
        text_words = temp_words
    syn = []
    syn_noun = []
    for i, val in enumerate(text_words):
        syn_noun.extend(wn.synsets(val, pos=wn.NOUN))

    for i, val in enumerate(text_words):
        syn.extend(wn.synsets(val))

    output_lesk = {}
    output_noun = {}
    context_words = []

    for i, val in enumerate(syn):
        words = word_tokenize(val.definition())
        context_words.extend(words)

    context_words = list(set(context_words) - set(stop_words))

    for word in list(wn.all_lemma_names(lang='eng')):
        if (len(word)== a+b and word.isalpha()) or (len(word) == a+b+1 and (word[a] == '-' or word[a] == '_') and word[0:a].isalpha() and word[a+1:a+b+1].isalpha()):
            sim_noun = 0
            sim = 0
            for synset in list(wn.synsets(word, pos=wn.NOUN)):
                temp = 0
                for i, val in enumerate(syn_noun):
                    temp = temp + synset.path_similarity(val)
                sim_noun = max(temp, sim_noun)

            for synset in list(wn.synsets(word)):
                def_words = word_tokenize(synset.definition())
                def_words = list(set(def_words) - set(stop_words))
                temp1 = set(text_words).intersection(def_words)
                temp2 = set(context_words).intersection(def_words)
                temp1 = len(temp1) / len(text_words)
                temp2 = len(temp2) / len(context_words)
                t = 0.75 * temp1 + 0.25 * temp2
                sim = max(t, sim)
            if sim > 0:
                output_lesk[word] = sim
            if sim_noun > 0:
                output_noun[word] = sim_noun
    nouns = sorted(output_noun, key=output_noun.__getitem__, reverse=True)
    lesk = sorted(output_lesk, key=output_lesk.__getitem__, reverse=True)

    max_nouns = 0
    for w in nouns:
        if output_noun[w] > max_nouns:
            max_nouns = output_noun[w]

    for key, value in output_noun.items():
        output_noun[key] = value / max_nouns

    for w in lesk:
        if not (w in output_noun):
            output_noun[w] = 0
        output_lesk[w] += output_noun[w]
    rank = list(sorted(output_lesk, key=output_lesk.__getitem__, reverse=True))
    final_rank=[]

    for w in rank:
        if w[a]=='_' or w[a]=='-':
            p=w[:a]+w[a+1:]
            final_rank.append(p)
            rank.remove(w)
        
    rank1= final_rank + rank
    return rank1[:1000]


In [77]:


def verb_fn(sent,a,b):
    text_words = word_tokenize(sent)
    stop_words = set(stopwords.words("english"))
    temp_words = list(set(text_words) - set(stop_words))
    if len(temp_words) != 0:
        text_words = temp_words
    syn = []
    syn_verb = []
    for i, val in enumerate(text_words):
        syn_verb.extend(wn.synsets(val, pos=wn.VERB))

    for i, val in enumerate(text_words):
        syn.extend(wn.synsets(val))

    output_lesk = {}
    output_verb = {}
    context_words = []

    for i, val in enumerate(syn):
        words = word_tokenize(val.definition())
        context_words.extend(words)

    context_words = list(set(context_words) - set(stop_words))

    for word in list(wn.all_lemma_names(lang='eng')):
        if (len(word)== a+b and word.isalpha()) or (len(word) == a+b+1 and (word[a] == '-' or word[a] == '_') and word[0:a].isalpha() and word[a+1:a+b+1].isalpha()):
            sim_verb = 0
            sim = 0
            for synset in list(wn.synsets(word, pos=wn.VERB)):
                temp = 0
                for i, val in enumerate(syn_verb):
                    temp = temp + synset.path_similarity(val)
                sim_verb = max(temp, sim_verb)

            for synset in list(wn.synsets(word)):
                def_words = word_tokenize(synset.definition())
                def_words = list(set(def_words) - set(stop_words))
                temp1 = set(text_words).intersection(def_words)
                temp2 = set(context_words).intersection(def_words)
                temp1 = len(temp1) / len(text_words)
                temp2 = len(temp2) / len(context_words)
                t = 0.75 * temp1 + 0.25 * temp2
                sim = max(t, sim)
            if sim > 0:
                output_lesk[word] = sim
            if sim_verb > 0:
                output_verb[word] = sim_verb
    verbs = sorted(output_verb, key=output_verb.__getitem__, reverse=True)
    lesk = sorted(output_lesk, key=output_lesk.__getitem__, reverse=True)

    max_verbs = 0
    for w in verbs:
        if output_verb[w] > max_verbs:
            max_verbs = output_verb[w]

    for key, value in output_verb.items():
        output_verb[key] = value / max_verbs

    for w in lesk:
        if not (w in output_verb):
            output_verb[w] = 0
        output_lesk[w] += output_verb[w]
    rank = list(sorted(output_lesk, key=output_lesk.__getitem__, reverse=True))

    final_rank=[]

    for w in rank:
        if w[a]=='_' or w[a]=='-':
            p=w[:a]+w[a+1:]
            final_rank.append(p)
            rank.remove(w)
        
    rank1= final_rank + rank
    return rank1[:1000]



In [78]:

def adj_fn(sent,a,b):
    text_words = word_tokenize(sent)
    stop_words = set(stopwords.words("english"))
    temp_words = list(set(text_words) - set(stop_words))
    if len(temp_words) != 0:
        text_words = temp_words
    syn = []
    syn_adj = []
    for i, val in enumerate(text_words):
        syn_adj.extend(wn.synsets(val, pos=wn.ADJ))

    for i, val in enumerate(text_words):
        syn.extend(wn.synsets(val))

    output_lesk = {}
    output_adj = {}
    context_words = []

    for i, val in enumerate(syn):
        words = word_tokenize(val.definition())
        context_words.extend(words)

    context_words = list(set(context_words) - set(stop_words))

    for word in list(wn.all_lemma_names(lang='eng')):
        if (len(word)== a+b and word.isalpha()) or (len(word) == a+b+1 and (word[a] == '-' or word[a] == '_') and word[0:a].isalpha() and word[a+1:a+b+1].isalpha()):
            sim_adj = 0
            sim = 0
            for synset in list(wn.synsets(word, pos=wn.ADJ)):
                temp = 0
                for i, val in enumerate(syn_adj):
                    if synset.path_similarity(val):
                        temp = temp + synset.path_similarity(val)
                sim_adj = max(temp, sim_adj)

            for synset in list(wn.synsets(word)):
                def_words = word_tokenize(synset.definition())
                def_words = list(set(def_words) - set(stop_words))
                temp1 = set(text_words).intersection(def_words)
                temp2 = set(context_words).intersection(def_words)
                temp1 = len(temp1) / len(text_words)
                temp2 = len(temp2) / len(context_words)
                t = 0.75 * temp1 + 0.25 * temp2
                sim = max(t, sim)
            if sim > 0:
                output_lesk[word] = sim
            if sim_adj > 0:
                output_adj[word] = sim_adj
    adjs = sorted(output_adj, key=output_adj.__getitem__, reverse=True)
    lesk = sorted(output_lesk, key=output_lesk.__getitem__, reverse=True)

    max_adjs = 0
    for w in adjs:
        if output_adj[w] > max_adjs:
            max_adjs = output_adj[w]

    for key, value in output_adj.items():
        output_adj[key] = value / max_adjs

    for w in lesk:
        if not (w in output_adj):
            output_adj[w] = 0
        output_lesk[w] += output_adj[w]
    rank = list(sorted(output_lesk, key=output_lesk.__getitem__, reverse=True))

    final_rank=[]

    for w in rank:
        if w[a]=='_' or w[a]=='-':
            p=w[:a]+w[a+1:];
            final_rank.append(p);
            rank.remove(w);
        
    rank1= final_rank + rank;
    return rank1[:1000]



In [79]:
def adv_fn(sent,a,b):
    text_words = word_tokenize(sent)
    stop_words = set(stopwords.words("english"))
    temp_words = list(set(text_words) - set(stop_words))
    if len(temp_words) != 0:
        text_words = temp_words
    syn = []
    syn_adv = []
    for i, val in enumerate(text_words):
        syn_adv.extend(wn.synsets(val, pos=wn.ADV))

    for i, val in enumerate(text_words):
        syn.extend(wn.synsets(val))

    output_lesk = {}
    output_adv = {}
    context_words = []

    for i, val in enumerate(syn):
        words = word_tokenize(val.definition())
        context_words.extend(words)

    context_words = list(set(context_words) - set(stop_words))

    for word in list(wn.all_lemma_names(lang='eng')):
        if (len(word)== a+b and word.isalpha()) or (len(word) == a+b+1 and (word[a] == '-' or word[a] == '_') and word[0:a].isalpha() and word[a+1:a+b+1].isalpha()):
            sim_adv = 0
            sim = 0
            for synset in list(wn.synsets(word, pos=wn.ADV)):
                temp = 0
                for i, val in enumerate(syn_adv):
                    if synset.path_similarity(val):
                        temp = temp + synset.path_similarity(val)
                sim_adv = max(temp, sim_adv)

            for synset in list(wn.synsets(word)):
                def_words = word_tokenize(synset.definition())
                def_words = list(set(def_words) - set(stop_words))
                temp1 = set(text_words).intersection(def_words)
                temp2 = set(context_words).intersection(def_words)
                temp1 = len(temp1) / len(text_words)
                temp2 = len(temp2) / len(context_words)
                t = 0.75 * temp1 + 0.25 * temp2
                sim = max(t, sim)
            if sim > 0:
                output_lesk[word] = sim
            if sim_adv > 0:
                output_adv[word] = sim_adv
    advs = sorted(output_adv, key=output_adv.__getitem__, reverse=True)
    lesk = sorted(output_lesk, key=output_lesk.__getitem__, reverse=True)

    max_advs = 0
    for w in advs:
        if output_adv[w] > max_advs:
            max_advs = output_adv[w]

    for key, value in output_adv.items():
        output_adv[key] = value / max_advs

    for w in lesk:
        if not (w in output_adv):
            output_adv[w] = 0
        output_lesk[w] += output_adv[w]
    rank = list(sorted(output_lesk, key=output_lesk.__getitem__, reverse=True))

    final_rank=[]

    for w in rank:
        if w[a]=='_' or w[a]=='-':
            p=w[:a]+w[a+1:]
            final_rank.append(p)
            rank.remove(w)
        
    rank1= final_rank + rank
    return rank1[:1000]

In [80]:



def dont_know_fn(sent, a, b):
    """
    This function takes a sentence, a value 'a', and a value 'b' as input and performs word similarity calculations using WordNet.
    It returns a ranked list of words based on their similarity scores.

    Parameters:
    - sent (str): The input sentence.
    - a (int): The first index for filtering words based on length and character conditions.
    - b (int): The second index for filtering words based on length and character conditions.

    Returns:
    - rank1[:1000] (list): A ranked list of words based on their similarity scores, limited to the top 1000 words.
    """

    text_words = word_tokenize(sent)
    stop_words = set(stopwords.words("english"))
    temp_words = list(set(text_words) - set(stop_words))
    if len(temp_words) != 0:
        text_words = temp_words
    syn = []
    for i, val in enumerate(text_words):
        syn.extend(wn.synsets(val))

    syn_verb = []
    for i, val in enumerate(text_words):
        syn_verb.extend(wn.synsets(val, pos=wn.VERB))

    syn_noun = []
    for i, val in enumerate(text_words):
        syn_noun.extend(wn.synsets(val, pos=wn.NOUN))

    syn_adj = []
    for i, val in enumerate(text_words):
        syn_adj.extend(wn.synsets(val, pos=wn.ADJ))

    syn_adv = []
    for i, val in enumerate(text_words):
        syn_adv.extend(wn.synsets(val, pos=wn.ADV))


    flag = 0
    output_verb = {}
    output_noun = {}
    output_adj = {}
    output_adv = {}
    output_lesk = {}
    context_words = []

    for i, val in enumerate(syn):
        words = word_tokenize(val.definition())
        context_words.extend(words)

    context_words = list(set(context_words) - set(stop_words))

    for word in list(wn.all_lemma_names(lang='eng')):
        if (len(word)== a+b and word.isalpha()) or (len(word) == a+b+1 and (word[a] == '-' or word[a] == '_') and word[0:a].isalpha() and word[a+1:a+b+1].isalpha()):
            #print("hello")
            sim_verb = 0
            sim_noun = 0
            sim_adj = 0
            sim_adv = 0
            sim = 0
            for synset in list(wn.synsets(word, pos=wn.VERB)):
                temp = 0
                for i, val in enumerate(syn_verb):
                    sim_verb = sim_verb + synset.path_similarity(val)
                sim_verb = max(temp, sim_verb)
            for synset in list(wn.synsets(word, pos=wn.NOUN)):
                temp = 0
                for i, val in enumerate(syn_noun):
                    temp = temp + synset.path_similarity(val)
                sim_noun = max(temp, sim_noun)
            for synset in list(wn.synsets(word, pos=wn.ADJ)):
                temp = 0
                for i, val in enumerate(syn_adj):
                    if synset.path_similarity(val):
                        temp = temp + synset.path_similarity(val)
                sim_adj = max(temp, sim_adj)
            for synset in list(wn.synsets(word, pos=wn.ADV)):
                temp = 0
                for i, val in enumerate(syn_adv):
                    if synset.path_similarity(val):
                        temp = temp + synset.path_similarity(val)
                sim_adv = max(temp, sim_adv)
            for synset in list(wn.synsets(word)):
                def_words = word_tokenize(synset.definition())
                def_words = list(set(def_words) - set(stop_words))
                temp1 = set(text_words).intersection(def_words)
                temp2 = set(context_words).intersection(def_words)
                temp1 = len(temp1) / len(text_words)
                temp2 = len(temp2) / len(context_words)
                t = 0.75 * temp1 + 0.25 * temp2
                sim = max(t, sim)

            if sim > 0:
                output_lesk[word] = sim
            if sim_verb > 0:
                output_verb[word] = sim_verb
            if sim_noun > 0:
                output_noun[word] = sim_noun
            if sim_adj > 0:
                output_adj[word] = sim_adj
            if sim_adv > 0:
                output_adj[word] = sim_adv

    verbs = sorted(output_verb, key=output_verb.__getitem__, reverse=True)
    nouns = sorted(output_noun, key=output_noun.__getitem__, reverse=True)
    adjs = sorted(output_adj, key=output_adj.__getitem__, reverse=True)
    advs = sorted(output_adv, key=output_adv.__getitem__, reverse=True)
    lesk = sorted(output_lesk, key=output_lesk.__getitem__, reverse=True)
    max_verbs = 0
    for w in verbs:
        if output_verb[w] > max_verbs:
            max_verbs = output_verb[w]

    for key, value in output_verb.items():
        output_verb[key] = value / max_verbs

    max_nouns = 0
    for w in nouns:
        if output_noun[w] > max_nouns:
            max_nouns = output_noun[w]

    for key, value in output_noun.items():
        output_noun[key] = value / max_nouns

    max_adjs = 0
    for w in adjs:
        if output_adj[w] > max_adjs:
            max_adjs = output_adj[w]

    for key, value in output_adj.items():
        output_adj[key] = value / max_adjs

    max_advs = 0
    for w in advs:
        if output_adv[w] > max_advs:
            max_advs = output_adv[w]

    for key, value in output_adv.items():
        output_adv[key] = value / max_advs

    #tot_rank1 = dict(output_verb, **output_noun)
    tot_rank1 = {k: max(i for i in (output_verb.get(k), output_noun.get(k)) if i) for k in
                output_verb.keys() | output_noun}

    #tot_rank = dict(tot_rank1, **output_adj)
    tot_rank2 = {k: max(i for i in (tot_rank1.get(k), output_adj.get(k)) if i) for k in
                tot_rank1.keys() | output_adj}

    tot_rank = {k: max(i for i in (tot_rank2.get(k), output_adv.get(k)) if i) for k in
                tot_rank2.keys() | output_adv}

    for w in lesk:
        if not (w in tot_rank):
            tot_rank[w] = 0
        tot_rank[w] += output_lesk[w]

    rank = list(sorted(tot_rank, key=tot_rank.__getitem__, reverse=True))

    final_rank=[]

    for w in rank:
        if w[a]=='_' or w[a]=='-':
            p=w[:a]+w[a+1:]
            final_rank.append(p)
            rank.remove(w)
        
    rank1= final_rank + rank
    
    return rank1[:1000]